In [47]:
from math import*
import __main__
global PI
import os
import time
PI=float(acos(-1))

import random
import numpy as np
import multiprocessing as mp

# random.seed()

data_size = 10000;

arr = np.random.randint(1, 10, size=[data_size, 8])
data_input = arr.tolist()
datasets = data_input

def ssh(num):
#     global datasets
#     arr = np.random.randint(1, 10, size=[data_size, 8])
#     datasets = arr.tolist()
    ply_angle = datasets[num]
    
    bas_ply=[-60, -45, -30, -15, 0, 15, 30, 45, 60, 90]

    AA1=bas_ply[ply_angle[0]]
    AA2=bas_ply[ply_angle[1]]
    AA3=bas_ply[ply_angle[2]]
    AA4=bas_ply[ply_angle[3]]
    AA5=bas_ply[ply_angle[4]]
    AA6=bas_ply[ply_angle[5]]
    AA7=bas_ply[ply_angle[6]]
    AA8=bas_ply[ply_angle[7]]

    ### ply stacking sequence###
    AAA=[AA1/180.0*PI,AA2/180.0*PI,AA3/180.0*PI,AA4/180.0*PI,AA5/180.0*PI,AA6/180.0*PI,AA7/180.0*PI,AA8/180.0*PI,AA8/180.0*PI,AA7/180.0*PI,AA6/180.0*PI,AA5/180.0*PI,AA4/180.0*PI,AA3/180.0*PI,AA2/180.0*PI,AA1/180.0*PI]
    pi=3.14159265358979

    R=250.0   ##  radius##
    H=510.0   ##  Height##
    td=0.125  #layer thickness##

    TTT=[-td*8,-td*7,-td*6,-td*5,-td*4,-td*3,-td*2,-td*1,td*0,td*1,td*2,td*3,td*4,td*5,td*6,td*7,td*8]

    ###material property###

    E1=123550.0  
    E2=8707.9
    G12=5695.0
    miu12=0.31946

    miu21=miu12*E2/E1
    Q11=E1/(1-miu12*miu21)
    Q12=miu21*E1/(1-miu12*miu21)
    Q22=E2/(1-miu12*miu21)
    Q66=G12

    A11=0.0
    A12=0.0
    A22=0.0
    A66=0.0

    D11=0.0
    D12=0.0
    D22=0.0
    D66=0.0

    for i in range(0,16):
        A11=A11+(Q11*cos(AAA[i])**4+2*(Q12+2*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q22*sin(AAA[i])**4)*(TTT[i+1]-TTT[i])
        A12=A12+((Q11+Q22-4*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q12*(sin(AAA[i])**4+cos(AAA[i])**4))*(TTT[i+1]-TTT[i])
        A22=A22+(Q11*sin(AAA[i])**4+2*(Q12+2*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q22*cos(AAA[i])**4)*(TTT[i+1]-TTT[i])
        A66=A66+((Q11+Q22-2*Q12-2*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q66*(sin(AAA[i])**4+cos(AAA[i])**4))*(TTT[i+1]-TTT[i])
        D11=D11+(Q11*cos(AAA[i])**4+2*(Q12+2*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q22*sin(AAA[i])**4)*(TTT[i+1]**3-TTT[i]**3)/3.0
        D12=D12+((Q11+Q22-4*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q12*(sin(AAA[i])**4+cos(AAA[i])**4))*(TTT[i+1]**3-TTT[i]**3)/3.0
        D22=D22+(Q11*sin(AAA[i])**4+2*(Q12+2*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q22*cos(AAA[i])**4)*(TTT[i+1]**3-TTT[i]**3)/3.0
        D66=D66+((Q11+Q22-2*Q12-2*Q66)*sin(AAA[i])**2*cos(AAA[i])**2+Q66*(sin(AAA[i])**4+cos(AAA[i])**4))*(TTT[i+1]**3-TTT[i]**3)/3.0

    #####bianliang########

    D= 2*R
    L= H

    #xian
    a=[[A11,A12,0],[A12,A22,0],[0,0,A66]]
    b=[[0,0,0],[0,0,0],[0,0,0]]
    d=[[D11,D12,0],[D12,D22,0],[0,0,D66]]

    alpha=PI/L
    beta=2/D

    mm=50
    nn=50
    kmm=0
    knn=0
    kmm11=0
    knn11=0
    F=[[0 for col in range(nn)] for row in range(mm)]
    Fcr=1e16

    for m in range(1,mm+1):
        for n in range(1,nn+1):
            xi11=2*a[0][0]*(m*alpha)**2+2*a[2][2]*(n*beta)**2
            xi12=2*(a[0][1]+a[2][2])*m*alpha*n*beta
            xi13=4*a[0][1]*m*alpha/D-2*b[0][0]*(m*alpha)**3-2*(b[0][1]+2*b[2][2])*m*alpha*(n*beta)**2
            xi22=2*a[1][1]*(n*beta)**2+2*a[2][2]*(m*alpha)**2
            xi23=4*a[1][1]*n*beta/D-2*b[1][1]*(n*beta)**3-2*(b[0][1]+2*b[2][2])*(m*alpha)**2*n*beta
            xi33=4*(d[0][1]+2*d[2][2])*(m*alpha*n*beta)**2+8*a[1][1]/(D**2)\
            +2*d[0][0]*(m*alpha)**4+2*d[1][1]*(n*beta)**4-8*(b[1][1]*(n*beta)**2+b[0][1]*(m*alpha)**2)/D
            xi21=xi12
            xi31=xi13
            xi32=xi23
            det1=xi11*(xi22*xi33-xi32*xi23)-xi12*(xi21*xi33-xi31*xi23)+xi13*(xi21*xi32-xi31*xi22)
            det2=xi11*xi22-xi21*xi12
            Nx=det1/det2/(2*((m*alpha)**2))
            F[m-1][n-1]=Nx*PI*D
            if Fcr>F[m-1][n-1]:
              Fcr=F[m-1][n-1]
              kmm=m
              knn=2*n

    for m in range(1,mm+1):
        xi11=2*a[0][0]*(m*alpha)**2
        xi12=0
        xi13=4*a[0][1]*m*alpha/D-2*b[0][0]*(m*alpha)**3
        xi22=2*a[2][2]*(m*alpha)**2
        xi23=0
        xi33=8*a[1][1]/(D**2)+2*d[0][0]*(m*alpha)**4-8*b[0][1]*(m*alpha)**2/D
        xi21=xi12
        xi31=xi13
        xi32=xi23
        det1=xi11*(xi22*xi33-xi32*xi23)-xi12*(xi21*xi33-xi31*xi23)+xi13*(xi21*xi32-xi31*xi22)
        det2=xi11*xi22-xi21*xi12
        Nx=det1/det2/(2*((m*alpha)**2))
        Fn1=Nx*PI*D
        if Fcr>Fn1:
          Fcr=Fn1
          kmm=m
          knn=1

    return Fcr


# map async parallel 
# pool = mp.Pool(mp.cpu_count())
pool = mp.Pool(processes = 8)

start = time.time()
results_map = pool.map(ssh, range(data_size))
end = time.time() 
print(end - start) #0.0037827491760253906
# print(results_map)


27.717838048934937


# Data preprocessing

In [48]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

results_map_sort = sorted(results_map) # note that sorted change the sort but didn't change the original one
# print(results_map_sort)
median_value = results_map_sort[data_size//2]
print(median_value)

results_map_new = np.zeros((data_size,1))
# print(type(results_map))
output_scikit = np.zeros(data_size)

for i in range(data_size):
    if results_map[i] > median_value:
        results_map_new[i,:] = 1
        output_scikit[i] = 1
    else:
        results_map_new[i,:] = 0
        output_scikit[i] = 0
        
# print(results_map) # new results_map transfer to [0,1]
# print(datasets)
# type(datasets)
# type(results_map)

X = torch.FloatTensor(datasets)
Y = torch.FloatTensor(results_map_new)
# type(data_tensor)
#type(output_tensor)
print(X.size())
print(Y.size())
# print(X)
# print(Y)


print(type(datasets))
print(type(output_scikit))

597624.0460489461
torch.Size([10000, 8])
torch.Size([10000, 1])
<class 'list'>
<class 'numpy.ndarray'>


# Training by classification 

In [49]:
# from sklearn import svm
# from sklearn.model_selection import cross_val_score

# clf = svm.SVC(gamma=0.001,probability=True)
# X = datasets
# y = results_map

# clf.fit(X, y) 

# score_svm = cross_val_score(clf, X, y, scoring='recall_macro',cv=5)  
# print(score_svm)

# clf.predict([datasets[3]])
# clf.predict_proba([datasets[3]])

In [50]:
# scikit-learn MLP training 
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_mldata
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

# preprocessing data 

X_scale = preprocessing.scale(datasets)
# print(X_scale)

# sgd optimizer 
mlp = MLPClassifier(solver='sgd', activation='relu',alpha=1e-4, hidden_layer_sizes=(128, 128),
                    random_state=1, max_iter=1000,verbose=True)

# lbfgs - very slow 
# mlp = MLPClassifier(solver='lbfgs', activation='relu',alpha=1e-4,hidden_layer_sizes=(100,100),
#                     random_state=1,max_iter=50,verbose=10,learning_rate_init=.1)

# adam 
# mlp = MLPClassifier(solver='adam', activation='relu',alpha=1e-4,hidden_layer_sizes=(100,100),
#                     random_state=1,max_iter=50,verbose=10,learning_rate_init=.1)

# mlp.fit(X_scale, output_scikit) 
# print(mlp.classes_)
# print(mlp.n_layers_)
# print(mlp.n_iter_)
# print(mlp.loss_)
# print(mlp.out_activation_)
# print(mlp.n_outputs_)

score_mlp = cross_val_score(mlp, X_scale, output_scikit,cv=5)  
print(score_mlp)

# 100000 data ~ 93%
# 10000 data - 85%
# 100000 data using 256, 256 - 91%
# 500000 data (128,128) - 96% 精度  0.96623034 0.96747    0.96845    0.96848    0.96720967
# 50000 (128,128) [0.89191081 0.8818     0.8794     0.8831     0.86838684]
# 10000[0.82358821 0.8385     0.8275     0.8375     0.76788394]

Iteration 1, loss = 0.69578929
Iteration 2, loss = 0.68854120
Iteration 3, loss = 0.68179479
Iteration 4, loss = 0.67588315
Iteration 5, loss = 0.67055195
Iteration 6, loss = 0.66560179
Iteration 7, loss = 0.66119054
Iteration 8, loss = 0.65704214
Iteration 9, loss = 0.65323425
Iteration 10, loss = 0.64971791
Iteration 11, loss = 0.64647671
Iteration 12, loss = 0.64347127
Iteration 13, loss = 0.64071231
Iteration 14, loss = 0.63820508
Iteration 15, loss = 0.63583294
Iteration 16, loss = 0.63371811
Iteration 17, loss = 0.63169330
Iteration 18, loss = 0.62983295
Iteration 19, loss = 0.62812876
Iteration 20, loss = 0.62648560
Iteration 21, loss = 0.62501629
Iteration 22, loss = 0.62362740
Iteration 23, loss = 0.62227025
Iteration 24, loss = 0.62097799
Iteration 25, loss = 0.61970758
Iteration 26, loss = 0.61861665
Iteration 27, loss = 0.61744765
Iteration 28, loss = 0.61631135
Iteration 29, loss = 0.61526012
Iteration 30, loss = 0.61426927
Iteration 31, loss = 0.61322993
Iteration 32, los

Iteration 253, loss = 0.47670944
Iteration 254, loss = 0.47603120
Iteration 255, loss = 0.47552049
Iteration 256, loss = 0.47508498
Iteration 257, loss = 0.47431956
Iteration 258, loss = 0.47380532
Iteration 259, loss = 0.47311936
Iteration 260, loss = 0.47265855
Iteration 261, loss = 0.47209407
Iteration 262, loss = 0.47159746
Iteration 263, loss = 0.47099542
Iteration 264, loss = 0.47028143
Iteration 265, loss = 0.46972026
Iteration 266, loss = 0.46921049
Iteration 267, loss = 0.46859980
Iteration 268, loss = 0.46809055
Iteration 269, loss = 0.46741206
Iteration 270, loss = 0.46685562
Iteration 271, loss = 0.46642738
Iteration 272, loss = 0.46599608
Iteration 273, loss = 0.46529619
Iteration 274, loss = 0.46474265
Iteration 275, loss = 0.46413999
Iteration 276, loss = 0.46360484
Iteration 277, loss = 0.46318899
Iteration 278, loss = 0.46242914
Iteration 279, loss = 0.46192975
Iteration 280, loss = 0.46138355
Iteration 281, loss = 0.46086609
Iteration 282, loss = 0.46037235
Iteration 

Iteration 504, loss = 0.36242082
Iteration 505, loss = 0.36197711
Iteration 506, loss = 0.36167079
Iteration 507, loss = 0.36112779
Iteration 508, loss = 0.36111237
Iteration 509, loss = 0.36033722
Iteration 510, loss = 0.35997714
Iteration 511, loss = 0.35958728
Iteration 512, loss = 0.35979388
Iteration 513, loss = 0.35945987
Iteration 514, loss = 0.35886615
Iteration 515, loss = 0.35834669
Iteration 516, loss = 0.35835351
Iteration 517, loss = 0.35775115
Iteration 518, loss = 0.35751123
Iteration 519, loss = 0.35689819
Iteration 520, loss = 0.35667331
Iteration 521, loss = 0.35666786
Iteration 522, loss = 0.35619591
Iteration 523, loss = 0.35575792
Iteration 524, loss = 0.35523396
Iteration 525, loss = 0.35514054
Iteration 526, loss = 0.35457253
Iteration 527, loss = 0.35462330
Iteration 528, loss = 0.35393986
Iteration 529, loss = 0.35367554
Iteration 530, loss = 0.35321476
Iteration 531, loss = 0.35279262
Iteration 532, loss = 0.35256242
Iteration 533, loss = 0.35213790
Iteration 

Iteration 33, loss = 0.61417527
Iteration 34, loss = 0.61321808
Iteration 35, loss = 0.61243978
Iteration 36, loss = 0.61144525
Iteration 37, loss = 0.61058727
Iteration 38, loss = 0.60971794
Iteration 39, loss = 0.60885596
Iteration 40, loss = 0.60795624
Iteration 41, loss = 0.60714075
Iteration 42, loss = 0.60624028
Iteration 43, loss = 0.60543032
Iteration 44, loss = 0.60458994
Iteration 45, loss = 0.60374675
Iteration 46, loss = 0.60291459
Iteration 47, loss = 0.60208401
Iteration 48, loss = 0.60130721
Iteration 49, loss = 0.60046993
Iteration 50, loss = 0.59965154
Iteration 51, loss = 0.59885078
Iteration 52, loss = 0.59803752
Iteration 53, loss = 0.59722395
Iteration 54, loss = 0.59644009
Iteration 55, loss = 0.59563404
Iteration 56, loss = 0.59489530
Iteration 57, loss = 0.59408572
Iteration 58, loss = 0.59329151
Iteration 59, loss = 0.59250498
Iteration 60, loss = 0.59172935
Iteration 61, loss = 0.59095214
Iteration 62, loss = 0.59018954
Iteration 63, loss = 0.58940794
Iteratio

Iteration 284, loss = 0.46656067
Iteration 285, loss = 0.46604223
Iteration 286, loss = 0.46531203
Iteration 287, loss = 0.46479183
Iteration 288, loss = 0.46434173
Iteration 289, loss = 0.46368276
Iteration 290, loss = 0.46331843
Iteration 291, loss = 0.46271926
Iteration 292, loss = 0.46201995
Iteration 293, loss = 0.46174879
Iteration 294, loss = 0.46106532
Iteration 295, loss = 0.46049584
Iteration 296, loss = 0.46002670
Iteration 297, loss = 0.45951675
Iteration 298, loss = 0.45897695
Iteration 299, loss = 0.45853741
Iteration 300, loss = 0.45806258
Iteration 301, loss = 0.45750151
Iteration 302, loss = 0.45692672
Iteration 303, loss = 0.45651352
Iteration 304, loss = 0.45575077
Iteration 305, loss = 0.45530380
Iteration 306, loss = 0.45494085
Iteration 307, loss = 0.45435494
Iteration 308, loss = 0.45374946
Iteration 309, loss = 0.45351552
Iteration 310, loss = 0.45283067
Iteration 311, loss = 0.45239183
Iteration 312, loss = 0.45179855
Iteration 313, loss = 0.45111775
Iteration 

...

Iteration 405, loss = 0.40425863
Iteration 406, loss = 0.40353082
Iteration 407, loss = 0.40330238
Iteration 408, loss = 0.40274590
Iteration 409, loss = 0.40280020
Iteration 410, loss = 0.40205020
Iteration 411, loss = 0.40163706
Iteration 412, loss = 0.40106808
Iteration 413, loss = 0.40063715
Iteration 414, loss = 0.40032906
Iteration 415, loss = 0.39992808
Iteration 416, loss = 0.39939114
Iteration 417, loss = 0.39874201
Iteration 418, loss = 0.39837484
Iteration 419, loss = 0.39819087
Iteration 420, loss = 0.39767847
Iteration 421, loss = 0.39731411
Iteration 422, loss = 0.39677918
Iteration 423, loss = 0.39630130
Iteration 424, loss = 0.39636362
Iteration 425, loss = 0.39539025
Iteration 426, loss = 0.39504328
Iteration 427, loss = 0.39464015
Iteration 428, loss = 0.39419584
Iteration 429, loss = 0.39358973
Iteration 430, loss = 0.39366048
Iteration 431, loss = 0.39306602
Iteration 432, loss = 0.39266606
Iteration 433, loss = 0.39227285
Iteration 434, loss = 0.39182239
Iteration 

Iteration 655, loss = 0.31600640
Iteration 656, loss = 0.31514366
Iteration 657, loss = 0.31486794
Iteration 658, loss = 0.31511605
Iteration 659, loss = 0.31523830
Iteration 660, loss = 0.31436054
Iteration 661, loss = 0.31408651
Iteration 662, loss = 0.31357219
Iteration 663, loss = 0.31351964
Iteration 664, loss = 0.31320072
Iteration 665, loss = 0.31309697
Iteration 666, loss = 0.31262248
Iteration 667, loss = 0.31247838
Iteration 668, loss = 0.31204971
Iteration 669, loss = 0.31184765
Iteration 670, loss = 0.31149043
Iteration 671, loss = 0.31152675
Iteration 672, loss = 0.31108427
Iteration 673, loss = 0.31086691
Iteration 674, loss = 0.31071271
Iteration 675, loss = 0.31059456
Iteration 676, loss = 0.30997917
Iteration 677, loss = 0.31035014
Iteration 678, loss = 0.30971779
Iteration 679, loss = 0.30924649
Iteration 680, loss = 0.30931348
Iteration 681, loss = 0.30908264
Iteration 682, loss = 0.30847957
Iteration 683, loss = 0.30827020
Iteration 684, loss = 0.30801641
Iteration 

Iteration 185, loss = 0.51493159
Iteration 186, loss = 0.51440217
Iteration 187, loss = 0.51392282
Iteration 188, loss = 0.51339093
Iteration 189, loss = 0.51289515
Iteration 190, loss = 0.51232961
Iteration 191, loss = 0.51187014
Iteration 192, loss = 0.51138337
Iteration 193, loss = 0.51079040
Iteration 194, loss = 0.51026379
Iteration 195, loss = 0.50981667
Iteration 196, loss = 0.50913714
Iteration 197, loss = 0.50876499
Iteration 198, loss = 0.50827867
Iteration 199, loss = 0.50767961
Iteration 200, loss = 0.50715995
Iteration 201, loss = 0.50663830
Iteration 202, loss = 0.50611347
Iteration 203, loss = 0.50557256
Iteration 204, loss = 0.50498659
Iteration 205, loss = 0.50453505
Iteration 206, loss = 0.50387566
Iteration 207, loss = 0.50335969
Iteration 208, loss = 0.50290148
Iteration 209, loss = 0.50234684
Iteration 210, loss = 0.50180731
Iteration 211, loss = 0.50129130
Iteration 212, loss = 0.50077465
Iteration 213, loss = 0.50029266
Iteration 214, loss = 0.49970202
Iteration 

Iteration 435, loss = 0.39403141
Iteration 436, loss = 0.39401469
Iteration 437, loss = 0.39304148
Iteration 438, loss = 0.39318073
Iteration 439, loss = 0.39262646
Iteration 440, loss = 0.39211891
Iteration 441, loss = 0.39175445
Iteration 442, loss = 0.39140003
Iteration 443, loss = 0.39116210
Iteration 444, loss = 0.39076427
Iteration 445, loss = 0.39025831
Iteration 446, loss = 0.38975750
Iteration 447, loss = 0.38937309
Iteration 448, loss = 0.38903423
Iteration 449, loss = 0.38889337
Iteration 450, loss = 0.38823139
Iteration 451, loss = 0.38801143
Iteration 452, loss = 0.38781828
Iteration 453, loss = 0.38731008
Iteration 454, loss = 0.38687115
Iteration 455, loss = 0.38675760
Iteration 456, loss = 0.38616107
Iteration 457, loss = 0.38586144
Iteration 458, loss = 0.38531690
Iteration 459, loss = 0.38491356
Iteration 460, loss = 0.38454301
Iteration 461, loss = 0.38423385
Iteration 462, loss = 0.38406308
Iteration 463, loss = 0.38366678
Iteration 464, loss = 0.38327005
Iteration 

Iteration 57, loss = 0.59360498
Iteration 58, loss = 0.59283411
Iteration 59, loss = 0.59236077
Iteration 60, loss = 0.59129629
Iteration 61, loss = 0.59014916
Iteration 62, loss = 0.58937613
Iteration 63, loss = 0.58847161
Iteration 64, loss = 0.58784822
Iteration 65, loss = 0.58756885
Iteration 66, loss = 0.58669904
Iteration 67, loss = 0.58586686
Iteration 68, loss = 0.58499874
Iteration 69, loss = 0.58433383
Iteration 70, loss = 0.58356722
Iteration 71, loss = 0.58298345
Iteration 72, loss = 0.58238775
Iteration 73, loss = 0.58123554
Iteration 74, loss = 0.58181940
Iteration 75, loss = 0.58110377
Iteration 76, loss = 0.57968063
Iteration 77, loss = 0.57911078
Iteration 78, loss = 0.57821905
Iteration 79, loss = 0.57760236
Iteration 80, loss = 0.57682875
Iteration 81, loss = 0.57570619
Iteration 82, loss = 0.57500264
Iteration 83, loss = 0.57427978
Iteration 84, loss = 0.57372682
Iteration 85, loss = 0.57364677
Iteration 86, loss = 0.57249943
Iteration 87, loss = 0.57140596
Iteratio

In [52]:
# FEM rank VS DNN probability rank 

prob = mlp.predict_proba(X_scale[1])

NotFittedError: This MLPClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

# MLP regression 


In [ ]:
# # sgd optimizer 
# from sklearn.neural_network import MLPRegressor
# mlp_regress = MLPRegressor(solver='sgd', activation='relu',alpha=1e-4, hidden_layer_sizes=(128, 128),
#                     random_state=1, max_iter=1000,verbose=True)

# # lbfgs - very slow 
# # mlp = MLPClassifier(solver='lbfgs', activation='relu',alpha=1e-4,hidden_layer_sizes=(100,100),
# #                     random_state=1,max_iter=50,verbose=10,learning_rate_init=.1)

# # adam 
# # mlp = MLPClassifier(solver='adam', activation='relu',alpha=1e-4,hidden_layer_sizes=(100,100),
# #                     random_state=1,max_iter=50,verbose=10,learning_rate_init=.1)

# # mlp.fit(X_scale, output_scikit) 
# # print(mlp.classes_)
# # print(mlp.n_layers_)
# # print(mlp.n_iter_)
# # print(mlp.loss_)
# # print(mlp.out_activation_)
# # print(mlp.n_outputs_)
# X_Re = preprocessing.scale(datasets)
# Y_Re = preprocessing.scale(results_map)

# score_regress = cross_val_score(mlp_regress, X_Re, Y_Re, cv=5)  
# print(score_regress)
# # [0.97351663 0.97390639 0.97552892 0.97428681 0.97503282] for 500000 data with [128,128]
# # [0.85165701 0.83106467 0.82760219 0.86254636 0.8405507 ] for 50000 data 
# # [0.69529696 0.68323394 0.6818702  0.68295741 0.72217451] for 10000 data